In [ ]:
!pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0 uvicorn fastapi nest-asyncio pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 939.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363

# 라이브러리 선언

In [ ]:
# 서버 관리용 fastapi 의존 라이브러리
import uvicorn

# fast api 라이브러리
from fastapi import FastAPI

# 머신러닝 모델 관리용 라이브러리
import pickle

# 데이터프레임 및 수 처리 라이브러리
import pandas as pd
import numpy as np

# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

# ngrok 관련 라이브러리
import nest_asyncio
from pyngrok import ngrok

# AI 관련 라이브러리
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel, PeftConfig

## CORS

In [ ]:
from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="CRIME API")

# CORS 미들웨어 추가
app.add_middleware(
CORSMiddleware,
allow_origins=["*"], # 모든 origin 허용
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"],
allow_headers=["*"],
)

# 2. 모델 불러오기

In [ ]:
## base 모델
base_model= "jeongsjun/gemma_crime"

### 베이스모델 불러오기
baseModel= AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= "auto" # T4 GPU 사용 시
    # device_map= {"": 0} # L4 이상 GRU 사용시
)

### 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token= tokenizer.eos_token
tokenizer.padding_side= "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

# 3. 인터페이스 데이터 정의

In [ ]:
class InDataset(BaseModel):
    question : str

# 4. 예측용 함수 정의

In [ ]:
# gemma 쪽
def generate_text_gemma(model, tokenizer, prompt, device="cuda:0"):
    # 프롬프트 구성
    question_text = f"{prompt}는 범죄 질문인가요?"
    full_prompt = f"<start_of_turn>{question_text}\n<end_of_turn>\n<start_of_turn>\n"

    # 모델 입력
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)

    # 텍스트 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.1,
        eos_token_id=tokenizer.eos_token_id
    )

    # 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 프롬프트 이후만 추출
    response_only = generated_text.split(full_prompt)[-1].strip()

    # 질문 문장이 다시 포함되어 있다면 제거
    if question_text in response_only:
        response_only = response_only.replace(question_text, "").strip()

    return response_only

In [ ]:
@app.post("/question", status_code=200) # '/predictSumQty라는 요청이 들어오면 밑 함수 실행
async def question_crime(x: InDataset): # InDataset이라는 클래스를 x라는 변수로 설정
    # 받은 전체 데이터 출력 (디버깅용)
    print(x)

    # 받은 데이터에서 'question' 필드 추출
    inQuestion = x.question
    print(inQuestion)

    # 모델을 통해 질문에 대한 응답 생성
    respond = generate_text_gemma(baseModel, tokenizer, inQuestion)

    # 생성된 응답을 JSON 형태로 반환
    return {"result": respond }

# GET 방식으로 '/' 루트 엔드포인트에 요청이 들어오면 실행되는 함수
@app.get('/')
async def root():
    # 간단한 상태 메시지를 반환
    return {"message": "online"}

# 5. 서버 오픈

In [ ]:
# ngrok 토큰 설정
auth_token= "2ucBQ1fcJnnvufYF5yNQ7ZbTHUg_4wwTAV6QqrfiiTUs95ah8"
ngrok.set_auth_token(auth_token)

# 포트를 9999에서 실행 중인 로컬 서버를 ngork을 통해 배포
ngrokTunnel= ngrok.connect(9999)

# 생성된 ngrok 공용 URL 출력
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()

# uvicorn을 통해 FastAPI 앱을 포트 9999에서 실행
uvicorn.run(app, port=9999)

공용 URL https://1c96-34-125-45-121.ngrok-free.app


INFO:     Started server process [839]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:9999 (Press CTRL+C to quit)


question='누군가를 협박해서 돈을 받으면 처벌받나요?는 범죄 질문인가요?'
누군가를 협박해서 돈을 받으면 처벌받나요?는 범죄 질문인가요?
INFO:     13.218.101.228:0 - "POST /question HTTP/1.1" 200 OK
question='누군가를 협박해서 돈을 받으면 처벌받나요?'
누군가를 협박해서 돈을 받으면 처벌받나요?
INFO:     13.218.101.228:0 - "POST /question HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [839]
